In [1]:
import platform
from qlib.contrib.data.handler import check_transform_proc
import qlib
from pathlib import Path
from qlib import data, model
from qlib.constant import REG_CN  # 'cn'
from qlib.data import D
from qlib.data.filter import ExpressionDFilter,NameDFilter
from qlib.utils import init_instance_by_config
from qlib.config import C

path = "F:\qlib_bin" if platform.node() == 'dimi-pc' else "D:\qlib_bin"
qlib.init(provider_uri={'day': Path(path)}, region=REG_CN)
# calendar max and min
D.calendar(freq='day')[[-1,0]]

[27604:MainThread](2023-08-03 21:18:50,775) INFO - qlib.Initialization - [config.py:415] - default_conf: client.
[27604:MainThread](2023-08-03 21:18:51,232) INFO - qlib.Initialization - [__init__.py:74] - qlib successfully initialized based on client settings.
[27604:MainThread](2023-08-03 21:18:51,233) INFO - qlib.Initialization - [__init__.py:76] - data_path={'day': WindowsPath('F:/qlib_bin')}


array([Timestamp('2023-07-14 00:00:00'), Timestamp('2005-01-04 00:00:00')],
      dtype=object)

In [2]:
# 只看沪深两市
sh = NameDFilter(name_rule_re = 'SH',fstart_time='2020-01-01', fend_time='2023-07-14')
sz = NameDFilter(name_rule_re = 'SZ',fstart_time='2020-01-01', fend_time='2023-07-14')

# 设置股票池
# all_shsz = D.instruments(market='all', filter_pipe=[sh,sz])


In [26]:
infer_proc

[{'class': 'ProcessInf', 'kwargs': {}},
 {'class': 'ZScoreNorm',
  'kwargs': {'fit_start_time': '2020-01-01', 'fit_end_time': '2020-01-31'}},
 {'class': 'Fillna', 'kwargs': {}}]

In [37]:
# dataLoader to get data
"""
example_loader_config = {
    'feature': (['expr_1', 'expr_2', ... , 'expr_n'],
               ['name_1', 'name_2', ... , 'name_n']), # name list是经过_parse_fields_info()函数处理过才有的
    ...,
    'label':  (['expr_1', 'expr_2', ... , 'expr_n'],
               ['name_1', 'name_2', ... , 'name_n']),    
}
"""
start_time = "2020-01-01"
end_time = "2023-06-30"
fit_start_time = start_time
fit_end_time = "2020-01-31"

"""
factor15
rank(correlation(rank(high), rank(volume), 3)：分别对最高价成交价和成交量进行计算排序分位值，然后计算二者近3天的相关系数并取排名分位值，本质上就是分析近3天最高价和成交量的相关性。
-1 * sum(rank(correlation(rank(high), rank(volume), 3)), 3)：最后对近3日内的相关系数分位值进行累加并取负号。
"""

factor = (["-1*Sum(Rank(Corr(Rank($high),Rank($volumn),3)),3)"],
          ["factor15"])

factor = (["Corr(Rank($high),Rank($volumn),3)"],
          ["factor15"])
label = (["Ref($close, -2)/Ref($close, -1) - 1"], ["LABEL0"])

dataloader_config = {
    "class": "QlibDataLoader",
    "module_path": "qlib.data.dataset.loader",
    "kwargs": {
        "config": {
            "feature": factor,
            "label": label,
        },
        "filter_pipe": [sh,sz],
        "freq": 'day',
        "inst_processor": None,
        "swap_level": True,
    },
}

DEFAULT_LEARN_PROCESSORS = [
    {"class": "DropnaLabel"},
    {"class": "DropnaProcessor"},
    {"class": "CSZScoreNorm", "kwargs": {"fields_group": "label"}},
]
DEFAULT_INFER_PROCESSORS = [
    {"class": "ProcessInf", "kwargs": {}},
    {"class": "ZScoreNorm", "kwargs": {}},
    {"class": "Fillna", "kwargs": {}},
]
learn_proc = check_transform_proc(DEFAULT_LEARN_PROCESSORS,fit_start_time,fit_end_time)
infer_proc = check_transform_proc(DEFAULT_INFER_PROCESSORS,fit_start_time,fit_end_time)

# [{'class': 'DropnaLabel', 'kwargs': {}},
#  {'class': 'DropnaProcessor', 'kwargs': {}},
#  {'class': 'CSZScoreNorm', 'kwargs': {'fields_group': 'label'}}]
# [{'class': 'ProcessInf', 'kwargs': {}},
#  {'class': 'ZScoreNorm',
#   'kwargs': {'fit_start_time': '2020-01-01', 'fit_end_time': '2020-01-31'}},
#  {'class': 'Fillna', 'kwargs': {}}]

datahandler_config = {
    "class": "DataHandlerLP",
    "module_path": "qlib.data.dataset.handler",
    "kwargs":{
        "start_time":start_time,
        "end_time":end_time,
        "data_loader":dataloader_config,
        "learn_processors":DEFAULT_LEARN_PROCESSORS,
        "infer_processors":DEFAULT_INFER_PROCESSORS,
        "instruments":"csi300",

    }
}

dataset_config = {
    "class": "DatasetH",
    "module_path": "qlib.data.dataset",
    "kwargs":{
        "handler": datahandler_config,
        "segments": {
            "insample": (start_time, "2022-12-31"),
            "outsample": ("2023-01-01", end_time),
            },
    }

}

datahandler = init_instance_by_config(datahandler_config)
# dataset = init_instance_by_config(dataset_config)

# # features, example from SH0000001
# D.features(instruments=all_shsz, start_time='2020-01-01', end_time='2023-07-14', fields=['$close', '$factor', '$high']).head(50)


# mkt_csi300 = D.instruments(market='all')
# D.features(mkt_csi300, fields, start_time=start, end_time=end, freq='day')

# # dataLoader params
# features = ['$close', '$factor', '$high']
# features = ["adjclose", "amount", "change", "close", "factor", "high", "low", "open", "volume", "vwap", "daily_return"]


TypeError: __init__() missing 1 required positional argument: 'N'

In [36]:
datahandler.get_cols()

['factor15', 'LABEL0']

In [2]:
# 从calendar中读取 指定的频率和时间范围，查看数据的时间范围
start,end = '2018-01-01','2023-02-28'
cal_d = D.calendar(start_time=start, end_time=end, freq='day')
print(cal_d[:2])

# 初始化一个stook pool 从instruments中，指定 market 为文件名，读取个股code
mkt_csi300 = D.instruments(market='csi300')   # {'market': 'all', 'filter_pipe': []}
mkt_csi300_list = D.list_instruments(instruments=mkt_csi300, start_time=start, end_time=end, as_list=True)

# 有filter api 用来过滤数据
# expressionDFilter = ExpressionDFilter(rule_expression='$close>2000')
# instruments = D.instruments(market='csi300', filter_pipe=[expressionDFilter])
# D.list_instruments(instruments=instruments, start_time='2020-01-01', end_time='2020-09-25', as_list=True)


[Timestamp('2018-01-02 00:00:00') Timestamp('2018-01-03 00:00:00')]


In [34]:
# 在这里做一些测试
import qlib
from qlib.utils import get_callable_kwargs
# from qlib.data.dataset import processor
# from qlib.data.dataset.processor import Processor
from qlib.contrib.data.handler import check_transform_proc

start_time = "2020-01-01"
end_time = "2023-05-26"
train_start_time = start_time
train_end_time = "2021-12-31"
val_start_time = "2022-01-01"
val_end_time = "2022-12-31"
test_start_time = "2023-01-01"
test_end_time = end_time

DEFAULT_LEARN_PROCESSORS = [
    {"class": "DropnaLabel"},
    {"class": "CSZScoreNorm", "kwargs": {"fields_group": "label"}},
]
DEFAULT_INFER_PROCESSORS = [
    {"class": "ProcessInf", "kwargs": {}},
    {"class": "ZScoreNorm", "kwargs": {}},
    {"class": "Fillna", "kwargs": {}},
]

a = {"class":"ZScoreNorm",
     "kwargs":{
         "fit_start_time": start_time,
         "fit_end_time": end_time,
     }
}
ll = [a]
check_transform_proc(DEFAULT_INFER_PROCESSORS,start_time,end_time)
# isinstance(a, Processor)
# a,b = get_callable_kwargs({"class": "ZScoreNorm", "kwargs": {"aaa":"bbb"}},processor)


[{'class': 'ProcessInf', 'kwargs': {}},
 {'class': 'ZScoreNorm',
  'kwargs': {'fit_start_time': '2020-01-01', 'fit_end_time': '2023-05-26'}},
 {'class': 'Fillna', 'kwargs': {}}]

In [27]:
a

qlib.data.dataset.processor.ZScoreNorm